## 客观概率和主观概率

https://zhuanlan.zhihu.com/p/331650901?utm_psn=1742504026049118208

1. 客观概率和主观概率

- 主观概率是依赖于观测主体而存在的，如果将观测主体破坏掉，则主观概率也就消失。这一点和客观概率不同。客观概率不依赖于观测主体是否存在，它是客观地存在着。
- 一个概率是不是主观概率？看它是不是依赖一个观测主体。
- 主观概率，就是依赖于某个观测主体而存在的概率，它是条件概率，就是观测主体从无限过去到现在，接收到的全部有关观测客体的有关观测值的条件下的条件概率，这也被称之为后验概率。

2. 在实际应用中很可能是有多个观测主体的，
- 在多个观测主体接收到的信息流不同的时候，它们对于同一观测客体的概率分布，或者说认识，是不同的。


3. 一个观测主体，同时也是一个控制主体。
- 控制是在观测当中无法避免的行为，控制会影响观测

4. 观测过程理论是反对任何估计行为的，我们只要得到有关观测客体的后验分布，这就够了。
- [观点]：根据观测的结果要不要做什么控制，是根据分布来做决策的，而不应当是根据什么估计值来做决策。


## 贝叶斯推断（Bayesian inference）

$ p(\theta|x) = \frac{p(x|\theta)*p(\theta)}{p(x)}$

其中 $\theta $, 代表一种假设。$ x $代表一种观察结果。

称 $ p(\theta) $ 为先验概率(prior)，是在还没有观测$ x $ 的情况下，$\theta $ 自身的概率。

称 $ p(\theta|x) $ 为后验概率(posterior)，表示在观察到了$ x $ 的情况下，$\theta $ 的条件概率。

称 $ \L (x|\theta) = C*p(x|\theta) $ 为似然函数（likelihood），其中C为常数，因为似然函数的绝对数值没有意义。

称 $ p(x) $ 为证据因子(evidence)，有时也会称为边缘似然。

>关于 先验概率 和条件概率:
>
>先验概率：指决策者基于自己的知识、经验，对某事件在未来发生可能性大小的一种估计值。
>
>条件概率是指：已知某特定情况（或事件）发生的前提下，某事件在未来发生可能性的一种估计值，定义式P（A|B）＝P(AB)／P(B)，　要求P(B)＞0．

# 随机变量和分布

## 离散分布
### 伯努利分布（0-1分布）

### 二项分布 （n重伯努利分布）

### 多项分布 （多结果的二项分布）

## 连续分布
### 正态分布 （二项分布的极限）


## 使用波士顿房价的数据来看看正态分布

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns

import scipy.stats as st
import seaborn as sns

In [ ]:
train = pd.read_csv(u'dataset/c_13_houseprice.csv')
train.head(3)

In [ ]:
SP = train['SalePrice'].replace([np.inf, -np.inf], np.nan)

In [ ]:
type(SP)

绘制直方图+分布图

In [ ]:
def hist_dist(pd):
    f, ax_hist = plt.subplots(1, sharex=True)
    _, FD_bins = np.histogram(pd, bins="fd")
    bin_nr = min(len(FD_bins)-1, 50)
    sns.histplot(data=pd, ax=ax_hist, bins=bin_nr, stat="density", alpha=0.4, kde=True, kde_kws={"cut": 3})
    ax_hist.set_title("SP and distribution")


In [ ]:
hist_dist(SP)

另一种查看是否服从正态分布的可视化方法，通过概率图来观察，


In [ ]:
plt.figure()
res = st.probplot(train['SalePrice'], plot=plt)

把房价做对数变换后再看

In [ ]:
SP_log = np.log(SP)
res = st.probplot(SalePrice_log, plot=plt)
plt.figure()

 

In [ ]:
sns.kdeplot(SP_log)

In [ ]:
hist_dist(SP_log)

## 中间值和均值
表现＂中间值＂的统计名词：

　　a.均值:   mean，数列的算术平均值，反应了数列的集中趋势,等于有效数值的合除以有效数值的个数．
  
　　b.中位值: median，等于排序后中间位置的值，当数列长度为偶数时，取中间两个值的均值．
  
　　c.众数: 　mode，等于数列出现次数最多的数字，当多个数字都出现最高次数时，多个数字都是众数．

在统计学中，为了区分＂样本（抽样）概念＂和＂总体概念＂，将均值区分为样本均值和总体均值，而它们的实际大小是相同的．

## 总体方差
方差(variance)是衡量随机变量或一组数据时离散程度的度量.实际上是每个样本和均值的差的和的平方的总和。

$ \sigma^2 = \frac{\sum_{i=1}^N{(X_i-\mu)^2}}{N} $

公式解析：

　　1.因为和样本数无关，所以分母为样本数

　　2.累加每个值和均值差值的平方，对应于每个值相对于均值的偏差，对应于离散程度，

　　3.平方是对离散程度的加剧，同时能让差值总为正数，以符合偏差的概念意义

　　4.显然，如果所有数值都乘以n倍，总体方差会乘以 $n^2$ 倍



## 样本方差
样本方差是指总体各单位变量值与其算术平均数的离差平方的平均数。
要点：样本方差是总体方差的无偏估计量。（无偏性：估计量抽样分布的数学期望等于被估计的总体参数）问题：为什么计算总体方差分母是“n”而样本方差分母是“n-1”？答：因为样本方差要更好的估计总体方差，则其应该是总体方差的无偏估计量，即 $ E（ s^2 ）= \sigma^2  $。

链接：https://zhuanlan.zhihu.com/p/315504747

样本方差的意义是用来估计总体方差（统计术语：样本方差是对总体方差的无偏估计），样本方差计算公式和总体方差不同:

$ s^2 = \frac{\sum_{i=1}^N{(X_i-\mu)^2}}{N-1} $